### Connect to the database

In [24]:
import sqlite3
import geopandas as gpd
conn = sqlite3.connect('../max-experiments/itinerary-scraping/journeys.db')
cursor = conn.cursor()

# display all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('journeys',), ('stops',), ('outages',)]


In [25]:
cursor.execute('SELECT * FROM outages')
outages = cursor.fetchall()

cursor.execute('SELECT * FROM journeys')
journeys = cursor.fetchall()

In [26]:
import pandas as pd
import matplotlib.pyplot as plt

outages_df = pd.DataFrame(outages, columns=['outage_id', 'stop_id', 'effect', 'updated_at', 'outage_data'])

In [27]:
# map outages to stops
cursor.execute('SELECT * FROM stops')
stops = cursor.fetchall()

stops_df = pd.DataFrame(stops, columns=['stop_id', 'stop_name', 'stop_coords', 'stop_acc'])  
stops_df

# merge outages_df with stops_df
merged_df = pd.merge(outages_df, stops_df, on='stop_id')


In [28]:
merged_df

,outage_id,stop_id,effect,updated_at,outage_data,stop_name,stop_coords,stop_acc
0,828,stop_point:tcl:SP:30205,unavailable,2024-05-25T17:28:00+02:00,"{""cause"": {""label"": ""Acc\u00e8s ferm\u00e9 pou...",Gorge de Loup,"{""lon"": ""4.805544"", ""lat"": ""45.766428""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h..."
1,8201,stop_point:tcl:SP:30205,unavailable,2024-05-26T05:33:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Gorge de Loup,"{""lon"": ""4.805544"", ""lat"": ""45.766428""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h..."
2,822,stop_point:tcl:SP:30205,unavailable,2024-05-29T05:08:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Gorge de Loup,"{""lon"": ""4.805544"", ""lat"": ""45.766428""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h..."
3,8202,stop_point:tcl:SP:30205,unavailable,2024-06-04T05:08:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Gorge de Loup,"{""lon"": ""4.805544"", ""lat"": ""45.766428""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h..."
4,8203,stop_point:tcl:SP:30205,unavailable,2024-06-04T08:38:00+02:00,"{""cause"": {""label"": ""Vandalisme""}, ""effect"": {...",Gorge de Loup,"{""lon"": ""4.805544"", ""lat"": ""45.766428""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h..."
...,...,...,...,...,...,...,...,...
178,7301,stop_point:tcl:SP:30187,unavailable,2024-05-29T17:53:00+02:00,"{""cause"": {""label"": ""Vandalisme""}, ""effect"": {...",Laennec,"{""lon"": ""4.886328"", ""lat"": ""45.738716""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h..."
179,241,stop_point:tcl:SP:42743,unavailable,2024-05-31T05:08:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Hôtel de Ville L. Pradel,"{""lon"": ""4.836091"", ""lat"": ""45.767309""}","[""has_wheelchair_boarding""]"
180,7601,stop_point:tcl:SP:30194,unavailable,2024-05-31T05:18:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Sans Souci,"{""lon"": ""4.864748"", ""lat"": ""45.747747""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h..."
181,761,stop_point:tcl:SP:30194,unavailable,2024-06-02T02:28:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Sans Souci,"{""lon"": ""4.864748"", ""lat"": ""45.747747""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h..."


In [29]:
# extract coordinates from stop_coords
merged_df['lon'] = merged_df['stop_coords'].apply(lambda x: x.split(',')[0][9:-1])
merged_df['lat'] = merged_df['stop_coords'].apply(lambda x: x.split(',')[1][9:-2])
merged_df

,outage_id,stop_id,effect,updated_at,outage_data,stop_name,stop_coords,stop_acc,lon,lat
0,828,stop_point:tcl:SP:30205,unavailable,2024-05-25T17:28:00+02:00,"{""cause"": {""label"": ""Acc\u00e8s ferm\u00e9 pou...",Gorge de Loup,"{""lon"": ""4.805544"", ""lat"": ""45.766428""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h...",4.805544,45.766428
1,8201,stop_point:tcl:SP:30205,unavailable,2024-05-26T05:33:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Gorge de Loup,"{""lon"": ""4.805544"", ""lat"": ""45.766428""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h...",4.805544,45.766428
2,822,stop_point:tcl:SP:30205,unavailable,2024-05-29T05:08:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Gorge de Loup,"{""lon"": ""4.805544"", ""lat"": ""45.766428""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h...",4.805544,45.766428
3,8202,stop_point:tcl:SP:30205,unavailable,2024-06-04T05:08:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Gorge de Loup,"{""lon"": ""4.805544"", ""lat"": ""45.766428""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h...",4.805544,45.766428
4,8203,stop_point:tcl:SP:30205,unavailable,2024-06-04T08:38:00+02:00,"{""cause"": {""label"": ""Vandalisme""}, ""effect"": {...",Gorge de Loup,"{""lon"": ""4.805544"", ""lat"": ""45.766428""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h...",4.805544,45.766428
...,...,...,...,...,...,...,...,...,...,...
178,7301,stop_point:tcl:SP:30187,unavailable,2024-05-29T17:53:00+02:00,"{""cause"": {""label"": ""Vandalisme""}, ""effect"": {...",Laennec,"{""lon"": ""4.886328"", ""lat"": ""45.738716""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h...",4.886328,45.738716
179,241,stop_point:tcl:SP:42743,unavailable,2024-05-31T05:08:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Hôtel de Ville L. Pradel,"{""lon"": ""4.836091"", ""lat"": ""45.767309""}","[""has_wheelchair_boarding""]",4.836091,45.767309
180,7601,stop_point:tcl:SP:30194,unavailable,2024-05-31T05:18:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Sans Souci,"{""lon"": ""4.864748"", ""lat"": ""45.747747""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h...",4.864748,45.747747
181,761,stop_point:tcl:SP:30194,unavailable,2024-06-02T02:28:00+02:00,"{""cause"": {""label"": ""Panne""}, ""effect"": {""labe...",Sans Souci,"{""lon"": ""4.864748"", ""lat"": ""45.747747""}","[""has_wheelchair_boarding"", ""has_elevator"", ""h...",4.864748,45.747747


In [30]:
tcl_lines = gpd.read_file('tcl_lines.json')
date_debut = tcl_lines['date_debut'] 
date_fin = tcl_lines['date_fin']
last_update = tcl_lines['last_update']
last_update_fme = tcl_lines['last_update_fme']
tcl_lines = tcl_lines.drop(columns=['date_debut', 'date_fin', 'last_update', 'last_update_fme'])

In [31]:
colors = []
for index, row in tcl_lines.iterrows():
    if row['ligne'] == 'D': # green line
        colors.append('#029C41')
    elif row['ligne'] == 'A':   # pink line
        colors.append('#E62E86')
    elif row['ligne'] == 'B':   # blue line
        colors.append('#0065B1')
    elif row['ligne'] == 'C':   # yellow line
        colors.append('#F48E06')
    else:   # furnicular line, light green
        colors.append("#93BF38")
        
tcl_lines['style'] = [
    {
        "color": colors[i],
        "weight": 4,
        "opacity": 1
    }
    for i in range(len(tcl_lines))
]

In [39]:
# plot outages on a heatmap
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

m = folium.Map(location=[45.75, 4.85], zoom_start=13)

# heatmap
heat_data = [[row['lat'], row['lon']] for idx, row in merged_df.iterrows()]
HeatMap(heat_data).add_to(m)

# marker cluster
marker_cluster = MarkerCluster().add_to(m)
for idx, row in merged_df.iterrows():
    stop_name = row['stop_name']
    outage_id = row['outage_id']
    folium.Marker([row['lat'], row['lon']], tooltip=f'Stop Name: {stop_name} <br>Outage ID: {outage_id}').add_to(marker_cluster)

# tcl metro lines
folium.GeoJson(tcl_lines, style_function=lambda x: x['properties']['style']).add_to(m)
    
m.save('outages_heatmap.html')